# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Import pprint
from pprint import pprint

### Store WeatherPy results into DataFrame

In [2]:
weather_csv_file = "../WeatherPy/output/city_weather_data.csv"
weather_df = pd.read_csv(weather_csv_file)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,georgetown,5.41,100.34,26.0,83.0,20.0,3.1,MY,1.601638e+09
1,arraial do cabo,-22.97,-42.03,26.0,78.0,23.0,5.7,BR,1.601638e+09
2,meiganga,6.52,14.30,27.3,64.0,93.0,2.6,CM,1.601639e+09
3,saint-philippe,-21.36,55.77,25.0,69.0,90.0,3.6,RE,1.601638e+09
4,ribeira grande,38.52,-28.70,21.0,73.0,75.0,3.6,PT,1.601638e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=gkey)

In [4]:
# Create a humidity heatmap layer
fig = gmaps.figure()

# Store Lat and Lng as locations
locations = weather_df[["Lat", "Lng"]].astype(float)

# Store humidity as weight
humidity = weather_df["Humidity"].astype(float)

# Retrieve max weather humidity for max intensity in heat map
max_intensity = max(weather_df["Humidity"])

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=max_intensity,
                                point_radius=2.5)

# Add heat layer on figure
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Find cities with ideal temperature

# Store previously created df into a new df
ideal_weather_df = weather_df

# A max temperature lower than 27 degrees celcius but higher than 21.
ideal_weather_df = ideal_weather_df.loc[(weather_df["Max Temp"] < 27) & (weather_df["Max Temp"] > 21)]

# Wind speed less than 4.5 m/s.
ideal_weather_df = ideal_weather_df.loc[weather_df["Wind Speed"] < 4.5]

# Zero cloudiness.
ideal_weather_df = ideal_weather_df.loc[weather_df["Cloudiness"] == 0]

# Drop any null values
ideal_weather_df = ideal_weather_df.dropna()

# Display all cities with ideal weather
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
165,cabo san lucas,22.89,-109.91,25.00,78.0,0.0,0.45,MX,1.601639e+09
217,guerrero negro,27.98,-114.06,22.06,68.0,0.0,1.17,MX,1.601639e+09
250,maneadero,31.72,-116.57,22.34,37.0,0.0,0.49,MX,1.601639e+09
310,altar,30.70,-111.82,23.29,22.0,0.0,0.15,MX,1.601639e+09
312,gazojak,41.19,61.40,22.83,19.0,0.0,2.99,TM,1.601639e+09
333,herat,34.50,62.00,26.00,14.0,0.0,3.10,AF,1.601639e+09
336,san patricio,19.22,-104.70,24.31,77.0,0.0,1.93,MX,1.601639e+09
403,baherden,38.44,57.43,24.59,17.0,0.0,3.56,TM,1.601639e+09
464,pakenham,-38.07,145.48,22.22,57.0,0.0,4.02,AU,1.601639e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named `hotel_df`
hotel_df = ideal_weather_df

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Check df
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
165,cabo san lucas,22.89,-109.91,25.00,78.0,0.0,0.45,MX,1.601639e+09,
217,guerrero negro,27.98,-114.06,22.06,68.0,0.0,1.17,MX,1.601639e+09,
250,maneadero,31.72,-116.57,22.34,37.0,0.0,0.49,MX,1.601639e+09,
310,altar,30.70,-111.82,23.29,22.0,0.0,0.15,MX,1.601639e+09,
312,gazojak,41.19,61.40,22.83,19.0,0.0,2.99,TM,1.601639e+09,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": gkey
}

# Use lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # Retrieve lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f'Retrieving Results for Index {index}: {row["City"]}')
    
    # make request and print url
    response = requests.get(base_url, params=params).json()
    
    #pprint(response)
    
    # Append generated hotel data into df with try-except
    try:
        print(f'The closest hotel is {response["results"][0]["name"]}')
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
          
    except:
        print("Missing field/result... skipping.")
          
    print("--------------------")

Retrieving Results for Index 165: cabo san lucas
The closest hotel is Cabo San Lucas
--------------------
Retrieving Results for Index 217: guerrero negro
The closest hotel is Guerrero Negro
--------------------
Retrieving Results for Index 250: maneadero
The closest hotel is El Maneadero
--------------------
Retrieving Results for Index 310: altar
The closest hotel is Altar Municipality
--------------------
Retrieving Results for Index 312: gazojak
The closest hotel is Gazadzhak
--------------------
Retrieving Results for Index 333: herat
The closest hotel is Aybak
--------------------
Retrieving Results for Index 336: san patricio
The closest hotel is San Patricio
--------------------
Retrieving Results for Index 403: baherden
The closest hotel is Baharly
--------------------
Retrieving Results for Index 464: pakenham
The closest hotel is Pakenham
--------------------


In [8]:
# Check dataframe's new column
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
165,cabo san lucas,22.89,-109.91,25.00,78.0,0.0,0.45,MX,1.601639e+09,Cabo San Lucas
217,guerrero negro,27.98,-114.06,22.06,68.0,0.0,1.17,MX,1.601639e+09,Guerrero Negro
250,maneadero,31.72,-116.57,22.34,37.0,0.0,0.49,MX,1.601639e+09,El Maneadero
310,altar,30.70,-111.82,23.29,22.0,0.0,0.15,MX,1.601639e+09,Altar Municipality
312,gazojak,41.19,61.40,22.83,19.0,0.0,2.99,TM,1.601639e+09,Gazadzhak
333,herat,34.50,62.00,26.00,14.0,0.0,3.10,AF,1.601639e+09,Aybak
336,san patricio,19.22,-104.70,24.31,77.0,0.0,1.93,MX,1.601639e+09,San Patricio
403,baherden,38.44,57.43,24.59,17.0,0.0,3.56,TM,1.601639e+09,Baharly
464,pakenham,-38.07,145.48,22.22,57.0,0.0,4.02,AU,1.601639e+09,Pakenham


In [12]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Store Lat and Lng as locations
locations = hotel_df[["Lat", "Lng"]]

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))